In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import openai
import os
from dotenv import load_dotenv
import json

In [7]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

RECIPE = {
    "title": "Whole Wheat Blueberry Brownies",
    "ingredients": [
        "flour", "sugar", "cocoa powder", "eggs", "milk", "butter", "baking powder"
    ],
    "steps": [
        "Combine dry ingredients in one bowl, and wet ingredients in another bowl except berries.",
        "Heat the oven to 350 degrees and spray an 8 x 8 x 2-inch pan with cooking spray.",
        "Combine wet and dry ingredients until smooth, then fold in berries.",
        "Pour into baking dish and bake for 20 25 minutes until a toothpick inserted in the center came out clean."
    ]
}

user_state = {
    "current_step": 0,
    "mistakes": []
}

user_history = {}

In [11]:
def answer_question(user_id, user_message):
    """Answer the user's question based on the recipe or general knowledge while keeping conversation history."""
    if user_id not in user_history:
        user_history[user_id] = [
            {"role": "system", "content": f"""
                You are a professional chef specializing in answering recipe-related questions.
                Here is the recipe data:
                {json.dumps(RECIPE, indent=2)}
                
                User's current step: {RECIPE["steps"][user_state['current_step']]}

                Answer questions based ONLY ON THESE RECIPE and general cooking knowledge. Shorten answers to at most 2 sentences.
            """}
        ]

    # Add user message to history
    user_history[user_id].append({"role": "user", "content": user_message})

    # Call OpenAI API with conversation history
    response = openai.ChatCompletion.create(
        model="gpt-4",
        api_key=OPENAI_API_KEY,
        messages=user_history[user_id]
    )

    bot_response = response["choices"][0]["message"]["content"].strip()

    # Add AI response to history
    user_history[user_id].append({"role": "assistant", "content": bot_response})

    # Limit message history to last 10 messages to save tokens
    user_history[user_id] = user_history[user_id][-10:]

    return bot_response

In [14]:
print(answer_question("1", "What did I ask just now?"))

You asked about what you are making right now.
